This notebook is for uploading leisure, shop, and amenities data to the database.

In [1]:
from dotenv import load_dotenv
import numpy as np
import os
import pymysql
import pandas as pd

In [2]:
load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT"))
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_DATABASE = os.getenv("DB_DATABASE")

In [3]:
df_sh = pd.read_csv("../../data_evaluation/osm_places/osm_places_shop_cleaned.csv") # OpenStreetMaps shop data
df_le = pd.read_csv("../../data_evaluation/osm_places/osm_places_leisure_cleaned.csv") # OpenStreetMaps leisure data
df_am = pd.read_csv("../../data_evaluation/osm_places/osm_places_amenity_cleaned.csv") # OpenStreetMaps amenity data
df_joined = pd.read_csv("places.csv")

In [4]:
# Filter out amenities that don't have google photos data yet
df_sh = df_sh[df_sh['id'].isin(df_joined['id'])]
df_le = df_le[df_le['id'].isin(df_joined['id'])]
df_am = df_am[df_am['id'].isin(df_joined['id'])]

In [5]:
# Top 10 cuisines, will be added as a variable
cuisines = [
    'pizza',
    'chinese',
    'coffee_shop',
    'mexican',
    'italian',
    'burger',
    'donut',
    'sandwich',
    'japanese',
    'american'
]

df_am['cuisine'] = df_am['cuisine'].astype(str) 

for cuisine in cuisines:
    df_am[cuisine] = df_am['cuisine'].apply(lambda s: cuisine in s.split(';'))

In [10]:
df_le

,id,lat,lon,leisure,name,addr:city,addr:postcode,addr:state,opening_hours,website,addr:street,addr:housenumber,phone,wheelchair,email
0,357545993,40.873155,-73.914025,park,Baker Field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
1,357562455,40.548161,-74.126253,park,Great Kills Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
2,357570943,40.705658,-73.936249,park,Ramirez Playground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
4,357594521,40.701492,-73.852913,park,Victory Field,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
9,357607904,40.706511,-74.003373,park,South Street Seaport Historic District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,11669827679,40.542279,-74.146272,dance,Dance Sensations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
368,11689452337,40.812128,-73.838106,park,Ferry Point Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
369,11771869135,40.667834,-73.964954,garden,Rose Arc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
370,11864791548,40.591085,-74.130633,dance,Staten Island Ballet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN


In [99]:
am_sql_table = """CREATE TABLE amenities (
id BIGINT,
cuisine_pizza BOOLEAN,
cuisine_chinese BOOLEAN,
cuisine_coffee_shop BOOLEAN,
cuisine_mexican BOOLEAN,
cuisine_italian BOOLEAN,
cuisine_burger BOOLEAN,
cuisine_donut BOOLEAN,
cuisine_sandwich BOOLEAN,
cuisine_japanese BOOLEAN,
cuisine_american BOOLEAN,
diet_vegan ENUM('yes', 'no', 'unknown'),
drink_beer ENUM('yes', 'no', 'unknown'),
drink_tea ENUM('yes', 'no', 'unknown'),
drink_wine ENUM('yes', 'no', 'unknown'),
outdoor_seating ENUM('yes', 'no', 'unknown'),
wheelchair ENUM('yes', 'no', 'limited', 'unknown'),
PRIMARY KEY (id),
FOREIGN KEY (id) REFERENCES places(id)
);
"""

In [91]:
sh_sql_table = """CREATE TABLE shop (
id BIGINT,
drink_coffee ENUM('yes', 'no', 'unknown'),
outdoor_seating ENUM('yes', 'no', 'unknown'),
wheelchair ENUM('yes', 'no', 'limited', 'unknown'),
PRIMARY KEY (id),
FOREIGN KEY (id) REFERENCES places(id)
);
"""

In [11]:
le_sql_table = """CREATE TABLE leisure (
id BIGINT,
wheelchair ENUM('yes', 'no', 'limited', 'unknown'),
PRIMARY KEY (id),
FOREIGN KEY (id) REFERENCES places(id)
);
"""

In [7]:
try:
    # Establish a connection to the database
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_DATABASE
    )

        # Create a cursor object to interact with the database
    cursor = connection.cursor()
except pymysql.Error as e:
    print(f"Error connecting to the database: {e}")

In [100]:
# Create the amenities table
try:
    cursor.execute(am_sql_table)
except pymysql.MySQLError as e:
    print(f"Error: {e}")

In [93]:
# Create the amenities table
try:
    cursor.execute(sh_sql_table)
except pymysql.MySQLError as e:
    print(f"Error: {e}")

In [16]:
# Create the leisure table
try:
    cursor.execute(le_sql_table)
except pymysql.MySQLError as e:
    print(f"Error: {e}")

In [103]:
am_insert_query = """
INSERT INTO amenities (
id,
cuisine_pizza,
cuisine_chinese,
cuisine_coffee_shop,
cuisine_mexican,
cuisine_italian,
cuisine_burger,
cuisine_donut,
cuisine_sandwich,
cuisine_japanese,
cuisine_american,
diet_vegan,
drink_beer,
drink_tea,
drink_wine,
outdoor_seating,
wheelchair
)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

data = [] # Data to be used

for i, row in df_am.iterrows():
    data.append([row['id'], row['pizza'], row['chinese'], row['coffee_shop'], row['mexican'],
                 row['italian'], row['burger'], row['donut'], row['sandwich'], row['japanese'], 
                 row['american'], row['diet:vegan'], row['drink:beer'], 
                 row['drink:tea'], row['drink:wine'],
                 row['outdoor_seating'], row['wheelchair']])


try:
    cursor.executemany(am_insert_query, data)
except pymysql.MySQLError as e:
    print(f"Error: {e}")
    connection.rollback()

connection.commit()

In [94]:
sh_insert_query = """
INSERT INTO shop (
id,
drink_coffee,
outdoor_seating,
wheelchair
)
VALUES (%s,%s,%s,%s)
"""

data = [] # Data to be used

for i, row in df_sh.iterrows():
    data.append([row['id'], row['drink:coffee'],
                 row['outdoor_seating'], row['wheelchair']])

try:
    cursor.executemany(sh_insert_query, data)
except pymysql.MySQLError as e:
    print(f"Error: {e}")
    connection.rollback()

connection.commit()

In [17]:
sh_insert_query = """
INSERT INTO leisure (
id,
wheelchair
)
VALUES (%s,%s)
"""

data = [] # Data to be used

for i, row in df_le.iterrows():
    data.append([row['id'], row['wheelchair']])

try:
    cursor.executemany(sh_insert_query, data)
except pymysql.MySQLError as e:
    print(f"Error: {e}")
    connection.rollback()

connection.commit()

In [18]:
query = """
SELECT *
FROM leisure
"""
print(cursor.execute(query))
cursor.fetchmany(5)

247


((357545993, 'no'),
 (357562455, 'no'),
 (357570943, 'no'),
 (357594521, 'no'),
 (357607904, 'no'))

In [98]:
# Delete the table
delete_query = """
DROP TABLE amenities
"""

cursor.execute(delete_query)

0

In [15]:
# Delete the shop table
delete_query = """
DROP TABLE leisure
"""

cursor.execute(delete_query)

0

In [64]:
# Close the cursor and connection
if cursor:
    cursor.close()
if connection:
    connection.close()
    print("Database connection closed.")

Database connection closed.
